In [ ]:
# The underneath cell is for the function definition.
# This cell can be ignored

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
def organize_hum(filename):
    # Load the data from the excel files
    data = pd.read_excel(filename)

    round_list = []
    for a_farm in data.farm.unique():
        one_farm = data[data.farm == a_farm]
        a_vvvo = one_farm.vvvo.unique()[0]
        a_farm_around = one_farm.groupby('slg_round', as_index=False).agg(
                                                    hum_round_mean = pd.NamedAgg(column = 'hum_barn', aggfunc = 'mean'),
                                                    hum_round_max = pd.NamedAgg(column = 'hum_barn', aggfunc = 'max'),
                                                    hum_round_min = pd.NamedAgg(column = 'hum_barn', aggfunc = 'min'),
                                                    hum_round_std = pd.NamedAgg(column = 'hum_barn', aggfunc = 'std'),
                                                    hum_round_sum = pd.NamedAgg(column = 'hum_barn', aggfunc = 'sum'),
                                                    hum_round_num = pd.NamedAgg(column = 'hum_barn', aggfunc = 'count'),
                                                    hum_round_tl_num = pd.NamedAgg(column = 'hum_barn', aggfunc = lambda x: (x<45).sum()),
                                                    #hum_round_tl_perc = pd.NamedAgg(column = 'hum_barn', aggfunc = lambda x: (x<45).sum()/'count'),
                                                    hum_round_al_num = pd.NamedAgg(column = 'hum_barn', aggfunc = lambda x: ((x>=45) & (x<50)).sum()),
                                                    hum_round_ok_num = pd.NamedAgg(column = 'hum_barn', aggfunc = lambda x: ((x>=50) & (x<60)).sum()),
                                                    hum_round_ah_num = pd.NamedAgg(column = 'hum_barn', aggfunc = lambda x: ((x>=60) & (x<70)).sum()),
                                                    hum_round_th_num = pd.NamedAgg(column = 'hum_barn', aggfunc = lambda x: (x>=70).sum()),
                                                    hum_round_min_date = pd.NamedAgg(column = 'Date', aggfunc = 'min'),
                                                    hum_round_max_date = pd.NamedAgg(column = 'Date', aggfunc = 'max')
                                                    )
        a_farm_around['farm']=a_farm
        a_farm_around['vvvo']=a_vvvo
        round_list = round_list + [a_farm_around]
    
    my={'farm':[], 'slg_round':[], 'start_date':[], 'end_date':[]}
    for l in range(1, 25):
        if not 'lag_'+str(l) in my.keys():
            my['lag_'+str(l)]=[]
    for a_farm in data.farm.unique():
        sub = data[data.farm==a_farm]
        for a_round in sub.slg_round.unique():
            df = sub[sub.slg_round==a_round]
            start_date = df.start_date.unique()[0]
            end_date = df.end_date.unique()[0]
            my['farm'].append(a_farm)
            my['slg_round'].append(a_round)
            my['start_date'].append(start_date)
            my['end_date'].append(end_date)
            s = df['hum_barn'].interpolate(method='linear').values
            vals = sm.tsa.acf(s, nlags=24)
            kk = 1
            for val in vals[1:]:
                my['lag_'+str(kk)].append(val)
                kk+=1
    df_final = pd.DataFrame.from_dict(my)
    
    df_final = df_final[['farm', 'slg_round', 'start_date', 'end_date','lag_12', 'lag_24']]
    
    df_sta = pd.concat(round_list).reset_index().rename(columns = {'index': 'slg_round'})
    #print(df_sta)
    for col in ['hum_round_tl_num', 'hum_round_al_num', 'hum_round_ok_num', 'hum_round_ah_num', 'hum_round_th_num']:
        #new_col = col.strip('_')
        df_sta['_'.join(col.split('_')[0:3]) + '_perc'] = df_sta[col]/df_sta['hum_round_num']
    df_ff = df_sta[[ 'farm','vvvo','slg_round', 'hum_round_mean', 'hum_round_max', 'hum_round_min',
       'hum_round_std', 'hum_round_sum', 'hum_round_num', 'hum_round_tl_perc', 'hum_round_al_perc', 'hum_round_ok_perc',
       'hum_round_ah_perc', 'hum_round_th_perc']]
    ff = pd.merge(df_ff, df_final, on = ['farm', 'slg_round'], how = 'left')
    return ff

In [2]:
A= organize_hum('humidity_remove_outlier.xlsx')

C:\Users\20204093\Anaconda3\envs\tf-gpu\lib\site-packages\statsmodels\tsa\stattools.py:662: FutureWarning: fft=True will become the default after the release of the 0.12 release of statsmodels. To suppress this warning, explicitly set fft=False.
  warnings.warn(
C:\Users\20204093\Anaconda3\envs\tf-gpu\lib\site-packages\statsmodels\tsa\stattools.py:662: FutureWarning: fft=True will become the default after the release of the 0.12 release of statsmodels. To suppress this warning, explicitly set fft=False.
  warnings.warn(
C:\Users\20204093\Anaconda3\envs\tf-gpu\lib\site-packages\statsmodels\tsa\stattools.py:662: FutureWarning: fft=True will become the default after the release of the 0.12 release of statsmodels. To suppress this warning, explicitly set fft=False.
  warnings.warn(
C:\Users\20204093\Anaconda3\envs\tf-gpu\lib\site-packages\statsmodels\tsa\stattools.py:662: FutureWarning: fft=True will become the default after the release of the 0.12 release of statsmodels. To suppress this 

ValueError: The column label 'slg_round' is not unique.